In [ ]:
# Importamos las librerías necesarias

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import time
import string
import spacy
from spacy import displacy
from textacy.viz.termite import draw_termite_plot
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from keras.layers import Reshape, merge, Concatenate, Lambda, Average
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

In [ ]:
# Cargamos el dataframe (en local en mi caso)

df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df

### PRE-PROCESAMIENTO

In [ ]:
# Primero observamos que hay una categoría duplicada, así que las unimos

df['category'] = df['category'].map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

#-------------------------------------------------------------------------------------------------------------------------------

# La forma en que utilizaremos los titulares y la descripción es uniendo ambos strings y analizando sobre éste

df['text'] = df['headline'] + " " + df['short_description']

#-------------------------------------------------------------------------------------------------------------------------------

# Estas dos funciones limpian el texto, lo tokeniza, quita las stop words, las palabras de 2 o menos caracteres,
# los no alfanuméricos y lematiza

stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()
my_sw = ['make', 'amp',  'news','new' ,'time', 'u','s', 'photos',  'get', 'say']

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2 and token not in my_sw

def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text)    
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)
    
#-------------------------------------------------------------------------------------------------------------------------------

# Función para el cálculo de la polaridad basado en TextBlob

def polarity_txt(text):
    return TextBlob(text).sentiment[0]

# Creamos la nueva columna de polaridad

df['polarity'] = df['text'].apply(polarity_txt)

#-------------------------------------------------------------------------------------------------------------------------------

# Función para el cálculo de la subjetividad basado en TextBlob

def subj_txt(text):
    return  TextBlob(text).sentiment[1]

# Creamos la nueva columna de subjetividad

df['subjectivity'] = df['text'].apply(subj_txt)

#-------------------------------------------------------------------------------------------------------------------------------

# Función para calcular la proporción del texto limpio con respecto al texto normal

def len_text(text):
    if len(text.split())>0:
        return len(set(clean_txt(text).split()))/ len(text.split())
    else:
        return 0
    
# Creamos la nueva columna con len

df['len'] = df['text'].apply(len_text)

# RETO 1:
### ¿Se pueden catalogar las noticias con la descripción y los titulares? Compara tu clasificación con las categorías incluidas en el set de datos.

In [ ]:
# Creamos las clases para el transformer y la extracción de estadísticas del texto

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return [{'pos':  row['polarity'], 'sub': row['subjectivity'],  'len': row['len']} for _, row in data.iterrows()]
    
#-------------------------------------------------------------------------------------------------------------------------------

# Creamos el flujo de procesamiento

pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[

            # Esta parte preprocesa el texto plano según la función de limpieza de texto que definimos y crea matrices TF-IDF
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf', TfidfVectorizer( min_df =3, max_df=0.2, max_features=None, 
                    strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 10), use_idf=1,smooth_idf=1,sublinear_tf=1,
                    stop_words = None, preprocessor=clean_txt)),
            ])),

            # Aquí se extraen características de los meta datos
            ('stats', Pipeline([
                ('selector', ItemSelector(key=['polarity', 'subjectivity', 'len'])),
                ('stats', TextStats()),
                ('vect', DictVectorizer()),
            ])),

        ],

        # Los pesos en la unión de los transformers
        transformer_weights={
            'text': 0.9,
            'stats': 1.5,
        },
    ))
])

#-------------------------------------------------------------------------------------------------------------------------------

# Aplicamos el flujo de procesamiento a la base, con una validación del 20%

seed = 40
X = df[['text', 'polarity', 'subjectivity','len']]
y = df['category']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
pipeline.fit(x_train)

#-------------------------------------------------------------------------------------------------------------------------------

# Descargamos el modelo a utilizar y lo instanciamos

# Utilizamos el modelo grande de spacy, !python -m spacy download en_core_web_lg en caso de no tenerlo descargado
nlp = spacy.load('en_core_web_lg')

# Indicamos los encoders

X = df['text']
y = df['category']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
Y = np_utils.to_categorical(y)

# Creamos los vectores con tf-idf (Term Frequency-Inverse Document Frequency)

vectorizer = TfidfVectorizer( min_df =3, max_df=0.2, max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = None, preprocessor=clean_txt)

seed = 40
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed, stratify =y)
vectorizer.fit(x_train)

word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in x_train]

print(X_train_sequences[0])

MAX_SEQ_LENGHT=60

N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in x_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)

EMBEDDINGS_LEN = 300

embeddings_index = np.zeros((len(vectorizer.get_feature_names()) + 1, EMBEDDINGS_LEN))
for word, idx in word2idx.items():
    try:
        embedding = nlp.vocab[word].vector
        embeddings_index[idx] = embedding
    except:
        pass
      
#-------------------------------------------------------------------------------------------------------------------------------

# Inicializamos el modelo, lo entrenamos (con 5 épocas) y probamos su precisión

model = Sequential()
model.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    EMBEDDINGS_LEN
                    weights=[embeddings_index],
                    input_length=MAX_SEQ_LENGHT,
                    trainable=False))
model.add(LSTM(300, dropout=0.2))
model.add(Dense(len(set(y)), activation='softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train_sequences, y_train, 
          epochs=5, batch_size=128, verbose=1, 
          validation_split=0.1)
 
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

# RETO 2: 
### ¿Existen estilos de escritura asociados a cada categoría?

In [ ]:
# Función que calcula la riqueza léxica (entendida como la cantidad de palabras únicas sobre la cantidad de palabras totales)

def lexical_richness(text):
    try:
        vocabulary = sorted(np.unique(nltk.tokenize.word_tokenize(text)))
        return len(vocabulary)/len(nltk.tokenize.word_tokenize(text))
    except:
        return 0

# Creamos la nueva columna de riqueza léxica y excluimos los que no tenían título ni descripción (riqueza léxica 0)

df['lexical_richness'] = df['text'].apply(lexical_richness)
df = df[df['lexical_richness']!=0]

#-------------------------------------------------------------------------------------------------------------------------------

# Vectorizamos el texto (de nuevo utilizando tf-idf)

vectorizer = TfidfVectorizer( min_df =3, max_df=0.2, max_features=None, 
                    strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 1), use_idf=1,smooth_idf=1,sublinear_tf=1,
                    stop_words = None, preprocessor=clean_txt)

vectorizer.fit(df.category)

#-------------------------------------------------------------------------------------------------------------------------------

# Creamos el top de palabras

def create_tf_matrix(category):
    return vectorizer.transform(df[df.category == category].text)

def create_term_freq(matrix, cat):
    category_words = matrix.sum(axis=0)
    category_words_freq = [(word, category_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    return pd.DataFrame(list(sorted(category_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms', cat])

for cat in df.category.unique():
    df_right = create_term_freq(create_tf_matrix(cat), cat).head(5)
    if cat != 'CRIME':
        df_top5_words = df_top5_words.merge(df_right, how='outer')
    else:
        df_top5_words = df_right.copy()
        
df_top5_words.fillna(0, inplace=True )
df_top5_words.set_index('Terms', inplace=True)
df_top5_words.shape

df_2 = df_top5_words.copy()
df_norm = (df_2) / (df_2.max() - df_2.min())

In [ ]:
# Gráfico con las 5 palabras más usadas por categoría

draw_termite_plot(np.array(df_norm.values), df_top5_words.columns, df_top5_words.index, highlight_cols=[], save='reto_2_words')

In [ ]:
# Gráfico del sentimiento por categoría

plt.figure(figsize=(20,11))
ax = sns.boxplot(x="category", y="polarity", data=df)
ax.set_title('Sentimiento por categorías')
l = ax.set_xticklabels(ax.get_xticklabels(), rotation=60)
plt.savefig('reto_2_sentimiento')

In [ ]:
# Gráfico de la subjetividad por categoría

plt.figure(figsize=(20,10))
ax = sns.boxplot(x="category", y="subjectivity", data=df)
ax.set_title('Subjetividad por categorías')
l= ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.savefig('reto_2_subjetividad')

In [ ]:
# Gráfaico de la riqueza léxica por categoría

plt.figure(figsize=(20,10))
ax = sns.boxplot(x="category", y="lexical_richness", data=df)
ax.set_title('Riqueza léxica por categorías')
l= ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.savefig('reto_2_lexical_richness')

# RETO 3: 
### ¿Qué se puede decir de los autores a partir de los datos?

In [ ]:
# Creamos un dataframe agregando por autor (si aparecen junto a otros se considera como un autor diferente)

df['qty']= 1
df_authors = df.groupby(['authors']).agg({'polarity': 'mean', 
                                     'subjectivity': 'mean', 
                                     'len': 'mean',
                                     'lexical_richness':'mean',
                                     'qty':'count'}).reset_index()

df_authors.drop([0], axis=0, inplace =True)
df_authors.reset_index(drop=True, inplace=True)

In [ ]:
# Top autores más profílicos (con más noticias)

df_prolific = df_authors.sort_values(['qty'], ascending=False).head(10)

def plot_grouped_bar(df, title):
    barWidth = 0.25

    # Definimos las variables (la riqueza léxica se divide en 2 para estar por debajo de 0.5 y graficarse mejor junto a las otras)
    bars1 = df.polarity
    bars2 = df.subjectivity
    bars3 = df.lexical_richness/2

    # Definimos las posiciones de las barras en el eje x
    r1 = np.arange(len(bars1))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]
    

    # Definimos colores, etiquetas, etc
    plt.figure(figsize=(15,10))
    plt.bar(r1, bars1, color='#F08080', width=barWidth, edgecolor='white', label='Sentimiento')
    plt.bar(r2, bars2, color='#B0C4DE', width=barWidth, edgecolor='white', label='Subjetividad')
    plt.bar(r3, bars3, color='#BDECB6', width=barWidth, edgecolor='white', label='Riqueza léxica')

    # Demás configuraciones del gráfico como título, ejes, etc
    plt.title(title)
    plt.xlabel('group', fontweight='bold')
    plt.xticks([r + barWidth for r in range(len(bars1))], df.authors)
    plt.xticks(rotation=60)
    plt.legend()
    plt.savefig(f'reto_3_{title}')
    plt.show()

plot_grouped_bar(df_prolific, "Sentimiento, Subjetividad y Riqueza léxica de los autores más prolíficos")

In [ ]:
# Top 5 autores más positivos y top 5 autores más negativos

df_polarity = df_authors[df_authors.qty >200].sort_values(['polarity'], ascending=False).head(5).append(df_authors[df_authors.qty >200].sort_values(['polarity'], ascending=True).head(5).iloc[::-1])
plot_grouped_bar(df_polarity, "Sentimiento, Subjetividad y Riqueza léxica de los autores más positivos (Top 5 alto) y más negativos (Top 5 bajo) con más de 200 publicaciones")

In [ ]:
# Top 5 autores más subjetivos y top 5 autores más objetivos

df_subjectivity = df_authors[df_authors.qty >200].sort_values(['subjectivity'], ascending=False).head(5).append(df_authors[df_authors.qty >200].sort_values(['subjectivity'], ascending=True).head(5).iloc[::-1])
plot_grouped_bar(df_subjectivity, "Sentimiento, Subjetividad y Riqueza léxica de los autores más subjetivos (Top 5 alto) y más objetivos (Top 5 bajo) con más de 200 publicaciones")

In [ ]:
# Top 5 autores con más riqueza léxica y top 5 con menor riqueza léxica

df_lexical_richness = df_authors[df_authors.qty >200].sort_values(['lexical_richness'], ascending=False).head(5).append(df_authors[df_authors.qty >200].sort_values(['lexical_richness'], ascending=True).head(5).iloc[::-1])
plot_grouped_bar(df_lexical_richness, "Sentimiento, Subjetividad y Riqueza léxica de los autores con más riqueza léxica (Top 5 alto) y con menor (Top 5 bajo) con más de 200 publicaciones")

# RETO 4:
### Ahora, utilizando técnicas de aprendizaje no supervisado, trata de identificar temas, “protagonistas” u otras entidades de las noticias.

In [ ]:
# Tomamos como referencia para la extracción de entidades las 100 noticias más largas

df['n_c'] = df['text'].apply(len)
df_100 = df.sort_values(['n_c'], ascending= False).head(100)

# Para hacerlo más sencillo, creamos un string que suma los 100 textos

string_text = ''
for i in df_100['text']:
    string_text = string_text + ' ' + i
    
# Utilizamos de nuevo el modelo grande de spacy

nlp = spacy.load('en_core_web_lg')

# Creamos un dataframe con las entidades encontradas

doc = nlp(string_text)
entities = []
labels = []
position_start = []
position_end = []

for ent in doc.ents:
    entities.append(ent) # La entidad encontrada
    labels.append(ent.label_) # Etiqueta de la entidad
    position_start.append(ent.start_char) # Posición del caracter inicial de la entidad en el string completo
    position_end.append(ent.end_char) # Posición del caracter final de la entidad en el string completo
    
df_entities = pd.DataFrame({'Entities': entities, 'Labels':labels, 'Position_start':position_start, 'Position_end': position_end})

In [ ]:
# Explicación de todos los tipos de entidades encontradas

for i in df_entities['Labels'].unique():
    print(i, ': ', spacy.explain(f'{i}'))

# RETO 5:
### Basándote en el texto de la descripción corta, caracteriza este dataset.

In [ ]:
# Creamos un nuevo dataframe teniendo en cuenta solo la descripción corta

df_short_description = df[['short_description']]
df_short_description = df_short_description[df_short_description['short_description']!='']
print(f'El {len(df_short_description)/len(df)*100}% de las noticias tienen una descipción corta')
df_short_description

# Calculamos la polaridad (solo teniendo en cuenta la descripción corta) y calculamos sus terciles

df_short_description['polarity'] = df_short_description['short_description'].apply(polarity_txt)
tercil_polarity = pd.qcut(df_short_description['polarity'], 3, labels=['Low', 'Medium', 'High'])
df_short_description = df_short_description.assign(tercil_polarity=tercil_polarity.values)

# Calculamos la subjetividad (solo teniendo en cuenta la descripción corta) y calculamos sus terciles

df_short_description['subjectivity'] = df_short_description['short_description'].apply(subj_txt)
tercil_subjectivity = pd.qcut(df_short_description['subjectivity'], 3, labels=['Low', 'Medium', 'High'])
df_short_description = df_short_description.assign(tercil_subjectivity=tercil_subjectivity.values)

# Calculamos la riqueza léxica (solo teniendo en cuenta la descripción corta) y calculamos sus extremos

df_short_description['lexical_richness'] = df_short_description['short_description'].apply(lexical_richness)
two_lexical_richness = pd.qcut(df_short_description['lexical_richness'], 2, labels=['Medium', 'High'])
df_short_description = df_short_description.assign(two_lexical_richness=two_lexical_richness.values)

df_short_description

In [ ]:
# Tienden a ser más objetivas aquellas noticias con mayor riqueza léxica

by_two_lexical_richness_subjectivity = df_short_description.groupby('two_lexical_richness')['tercil_subjectivity'].value_counts(normalize=True)
by_two_lexical_richness_subjectivity.unstack().reindex(columns=['Low', 'Medium', 'High']).plot(kind='bar', title='Subjetividad por riqueza léxica', stacked=False)
plt.savefig(f'reto_5_subjetividad')

In [ ]:
# Tienden a ser más negativas aquellas noticias con mayor riqueza léxica

by_two_lexical_richness_polarity = df_short_description.groupby('two_lexical_richness')['tercil_polarity'].value_counts(normalize=True)
by_two_lexical_richness_polarity.unstack().reindex(columns=['Low', 'Medium', 'High']).plot(kind='bar', title='Sentimiento por riqueza léxica', stacked=False)
plt.savefig(f'reto_5_sentimiento')